In [135]:
# Import Python packages
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import json
import sys
import cachetools
from datetime import datetime
from datetime import date
import matplotlib.pyplot as plt 
import seaborn as sns
import math

# Import Snowflake modules
from snowflake.snowpark import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark import Window
from snowflake.snowpark.functions import col

# Clustering & Model
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score, train_test_split

# Scalers
from sklearn.preprocessing import StandardScaler

# Evaluation
from sklearn.metrics import silhouette_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

# find the best combination of model hyperparameters
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_score, cross_validate

# Getting Password,Username, Account
import getpass

In [136]:
# Get account credentials from a json file
with open("account.json") as f:
    data = json.load(f)
    username = data["username"]
    password = data["password"]
    account = data["account"]

# Specify connection parameters
connection_parameters = {
    "account": account,
    "user": username,
    "password": password,
    #"role": "ACCOUNTADMIN",
    #"warehouse": "tasty_ds_wh",
    #"database": "NGEE_ANN_POLYTECHNIC_FROSTBYTE_DATA_SHARE",
    #"schema": "analytics",
}

# Create Snowpark session
session = Session.builder.configs(connection_parameters).create()

In [137]:
order_header_df = session.table("NGEE_ANN_POLYTECHNIC_FROSTBYTE_DATA_SHARE.raw_pos.order_header")
order_detail_df = session.table("NGEE_ANN_POLYTECHNIC_FROSTBYTE_DATA_SHARE.raw_pos.order_detail")
menu_df = session.table("NGEE_ANN_POLYTECHNIC_FROSTBYTE_DATA_SHARE.raw_pos.menu")
truck_df = session.table("NGEE_ANN_POLYTECHNIC_FROSTBYTE_DATA_SHARE.raw_pos.truck")

In [138]:
order_header_df.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_ID"  |"TRUCK_ID"  |"LOCATION_ID"  |"CUSTOMER_ID"  |"DISCOUNT_ID"  |"SHIFT_ID"  |"SHIFT_START_TIME"  |"SHIFT_END_TIME"  |"ORDER_CHANNEL"  |"ORDER_TS"           |"SERVED_TS"  |"ORDER_CURRENCY"  |"ORDER_AMOUNT"  |"ORDER_TAX_AMOUNT"  |"ORDER_DISCOUNT_AMOUNT"  |"ORDER_TOTAL"  |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|22477378    |97          |3713.0         |NULL           |NULL           |78881       |16:00:00            |23:00:00          |NULL             |2020-04-

In [139]:
order_detail_df.show()

--------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_DETAIL_ID"  |"ORDER_ID"  |"MENU_ITEM_ID"  |"DISCOUNT_ID"  |"LINE_NUMBER"  |"QUANTITY"  |"UNIT_PRICE"  |"PRICE"  |"ORDER_ITEM_DISCOUNT_AMOUNT"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------
|349467439          |128946760   |121             |NULL           |0              |1           |20.0000       |20.0000  |NULL                          |
|349467440          |128946760   |122             |NULL           |1              |2           |12.0000       |24.0000  |NULL                          |
|349467441          |128946760   |123             |NULL           |2              |1           |11.0000       |11.0000  |NULL                          |
|349467442          |128946760   |125             |NULL           |3              

In [140]:
menu_df.show()

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"MENU_ID"  |"MENU_TYPE_ID"  |"MENU_TYPE"  |"TRUCK_BRAND_NAME"  |"MENU_ITEM_ID"  |"MENU_ITEM_NAME"    |"ITEM_CATEGORY"  |"ITEM_SUBCATEGORY"  |"COST_OF_GOODS_USD"  |"SALE_PRICE_USD"  |"MENU_ITEM_HEALTH_METRICS_OBJ"     |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|10001      |1               |Ice Cream    |Freezing Point      |10              |Lemonade            |Beverage         |Cold Option         |0.6500               |3.5000            |{                                  |
|           |                |             |                    |                |                    |                 

In [141]:
#Minimum order transaction by a truck
order_header_df.group_by("TRUCK_ID").agg(F.count("ORDER_ID").alias('count')).agg(F.min('count')).show()

----------------
|"MIN(COUNT)"  |
----------------
|8850          |
----------------



In [142]:
#order_header_df.drop_duplicates('ORDER_ID').group_by('ORDER_ID').agg(F.count('ORDER_ID').alias('nnn')).agg(F.sum('NNN')).show()

In [143]:
#order_detail_df.drop_duplicates('ORDER_ID').group_by('ORDER_ID').agg(F.count('ORDER_ID').alias('nnn')).agg(F.sum('NNN')).show()

In [144]:
order_truck = order_header_df.select('ORDER_ID', 'TRUCK_ID', 'ORDER_TS')
df = order_detail_df.join(order_truck, order_detail_df.ORDER_ID == order_truck.ORDER_ID, how = "left", lsuffix = "", rsuffix = "_01")

In [145]:
df.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_DETAIL_ID"  |"ORDER_ID"  |"MENU_ITEM_ID"  |"DISCOUNT_ID"  |"LINE_NUMBER"  |"QUANTITY"  |"UNIT_PRICE"  |"PRICE"  |"ORDER_ITEM_DISCOUNT_AMOUNT"  |"ORDER_ID_01"  |"TRUCK_ID"  |"ORDER_TS"           |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|798202497          |420427397   |133             |NULL           |0              |1           |6.0000        |6.0000   |NULL                          |420427397      |433         |2022-04-10 19:42:18  |
|798202503          |420427400   |131             |NULL           |0              |4           |13.0000       |52.0000  |NULL                          |420427400      |433         |202

In [146]:
df = df.with_column('ORDER_YEAR', F.year(F.col('ORDER_TS')))
df = df.with_column('ORDER_MONTH', F.month(F.col('ORDER_TS')))

In [147]:
#df.show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_DETAIL_ID"  |"ORDER_ID"  |"MENU_ITEM_ID"  |"DISCOUNT_ID"  |"LINE_NUMBER"  |"QUANTITY"  |"UNIT_PRICE"  |"PRICE"  |"ORDER_ITEM_DISCOUNT_AMOUNT"  |"ORDER_ID_01"  |"TRUCK_ID"  |"ORDER_TS"           |"ORDER_YEAR"  |"ORDER_MONTH"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|798202497          |420427397   |133             |NULL           |0              |1           |6.0000        |6.0000   |NULL                          |420427397      |433         |2022-04-10 19:42:18  |2022          |4              |
|798202503          |420427400   |131             |NULL     

In [148]:
#df.describe().show()

In [149]:
demand_df = df.group_by("TRUCK_ID", 'MENU_ITEM_ID', 'ORDER_YEAR', 'ORDER_MONTH').agg(F.sum("QUANTITY").alias('DEMAND'))
demand_df.sort('TRUCK_ID').show()

-------------------------------------------------------------------------
|"TRUCK_ID"  |"MENU_ITEM_ID"  |"ORDER_YEAR"  |"ORDER_MONTH"  |"DEMAND"  |
-------------------------------------------------------------------------
|1           |13              |2022          |4              |974       |
|1           |14              |2022          |6              |336       |
|1           |14              |2022          |4              |319       |
|1           |14              |2019          |2              |520       |
|1           |11              |2022          |11             |25        |
|1           |17              |2022          |8              |961       |
|1           |15              |2022          |7              |358       |
|1           |12              |2019          |3              |1769      |
|1           |13              |2022          |8              |979       |
|1           |13              |2022          |9              |942       |
--------------------------------------

In [150]:
unit_price = df.group_by("TRUCK_ID", 'MENU_ITEM_ID', 'ORDER_YEAR', 'ORDER_MONTH').agg(F.mean("unit_price").alias('unit_price'))
unit_price.sort('TRUCK_ID').show()

-----------------------------------------------------------------------------
|"TRUCK_ID"  |"MENU_ITEM_ID"  |"ORDER_YEAR"  |"ORDER_MONTH"  |"UNIT_PRICE"  |
-----------------------------------------------------------------------------
|1           |12              |2022          |3              |6.0000000000  |
|1           |11              |2022          |7              |6.0000000000  |
|1           |18              |2022          |6              |5.0000000000  |
|1           |14              |2022          |9              |2.0000000000  |
|1           |19              |2022          |8              |3.0000000000  |
|1           |10              |2022          |6              |3.5000000000  |
|1           |15              |2022          |3              |3.0000000000  |
|1           |15              |2022          |9              |3.0000000000  |
|1           |16              |2022          |9              |3.0000000000  |
|1           |19              |2022          |7              |3.

In [151]:
final_df = demand_df.join(unit_price, on= ['TRUCK_ID', 'MENU_ITEM_ID', 'ORDER_YEAR', 'ORDER_MONTH'], lsuffix = "", rsuffix = "_01")

In [152]:
truck_df.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"TRUCK_ID"  |"MENU_TYPE_ID"  |"PRIMARY_CITY"  |"REGION"    |"ISO_REGION"  |"COUNTRY"      |"ISO_COUNTRY_CODE"  |"FRANCHISE_FLAG"  |"YEAR"  |"MAKE"        |"MODEL"           |"EV_FLAG"  |"FRANCHISE_ID"  |"TRUCK_OPENING_DATE"  |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|3           |3               |San Mateo       |California  |CA            |United States  |US                  |1                 |2004    |Freightliner  |MT45 Utilimaster  |0          |2               |2021-10-01            |
|4           |4               |San Mateo       |California  |CA            |United State

In [153]:
#truck_df = truck_df.filter(F.col('COUNTRY') == 'United States')

In [154]:
truck = truck_df.with_column('LAST_DATE', F.iff(F.col("TRUCK_ID") == F.col('TRUCK_ID'), "2022-10-18", '0'))
truck = truck.withColumn("DAYS_OPENED", F.datediff("day", F.col("TRUCK_OPENING_DATE"), F.col('LAST_DATE')))
#truck = truck.select('TRUCK_ID', 'PRIMARY_CITY', 'REGION', 'COUNTRY', 'DAYS_OPENED')
truck = truck.select('TRUCK_ID','EV_FLAG','DAYS_OPENED')

truck.show()

------------------------------------------
|"TRUCK_ID"  |"EV_FLAG"  |"DAYS_OPENED"  |
------------------------------------------
|3           |0          |382            |
|4           |1          |655            |
|5           |1          |200            |
|6           |0          |1205           |
|7           |0          |655            |
|8           |1          |200            |
|9           |0          |839            |
|12          |0          |565            |
|13          |0          |474            |
|14          |0          |747            |
------------------------------------------



In [155]:
final_df = final_df.join(truck, on= ['TRUCK_ID'], lsuffix = "", rsuffix = "_01")

In [156]:
menu = menu_df.drop('MENU_ITEM_HEALTH_METRICS_OBJ')
menu.show()

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"MENU_ID"  |"MENU_TYPE_ID"  |"MENU_TYPE"  |"TRUCK_BRAND_NAME"  |"MENU_ITEM_ID"  |"MENU_ITEM_NAME"    |"ITEM_CATEGORY"  |"ITEM_SUBCATEGORY"  |"COST_OF_GOODS_USD"  |"SALE_PRICE_USD"  |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|10001      |1               |Ice Cream    |Freezing Point      |10              |Lemonade            |Beverage         |Cold Option         |0.6500               |3.5000            |
|10002      |1               |Ice Cream    |Freezing Point      |11              |Sugar Cone          |Dessert          |Cold Option         |2.5000               |6.0000            |
|10003      |1               |Ice Cream    |Freezing Point      |12             

In [157]:
menu = menu.with_column('TEMPERATURE_OPTION', F.when(F.col('ITEM_SUBCATEGORY') == 'Cold Option', 0).when(
    F.col('ITEM_SUBCATEGORY') == 'Warm Option', 1).otherwise(2))
menu = menu.select('MENU_ITEM_ID', 'MENU_TYPE_ID', 'TEMPERATURE_OPTION', 'COST_OF_GOODS_USD', 'ITEM_CATEGORY') #Add item category and ohe in pandas
menu.show()

--------------------------------------------------------------------------------------------------
|"MENU_ITEM_ID"  |"MENU_TYPE_ID"  |"TEMPERATURE_OPTION"  |"COST_OF_GOODS_USD"  |"ITEM_CATEGORY"  |
--------------------------------------------------------------------------------------------------
|23              |2               |1                     |7.0000               |Main             |
|24              |2               |0                     |0.5000               |Beverage         |
|25              |2               |0                     |0.5000               |Beverage         |
|26              |2               |0                     |0.7500               |Beverage         |
|27              |2               |0                     |2.2500               |Snack            |
|28              |2               |2                     |11.2500              |Main             |
|29              |2               |1                     |1.2500               |Snack            |
|31       

In [158]:
final_df = final_df.join(menu, on= ['MENU_ITEM_ID'], lsuffix = "", rsuffix = "_01")

In [159]:
#final_df.show()

In [160]:
import feature_engine
from feature_engine.encoding import OneHotEncoder

final_df = final_df.to_pandas()
ohe_enc = OneHotEncoder(
    top_categories=None,
    drop_last=False,
    variables= ['ITEM_CATEGORY'])  # to return k-1, false to return k


ohe_enc.fit(final_df)
final_df = ohe_enc.transform(final_df)

In [161]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60291 entries, 0 to 60290
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   MENU_ITEM_ID            60291 non-null  int16  
 1   TRUCK_ID                60291 non-null  int16  
 2   ORDER_YEAR              60291 non-null  int16  
 3   ORDER_MONTH             60291 non-null  int8   
 4   DEMAND                  60291 non-null  int64  
 5   UNIT_PRICE              60291 non-null  object 
 6   EV_FLAG                 60291 non-null  int8   
 7   DAYS_OPENED             60291 non-null  int32  
 8   MENU_TYPE_ID            60291 non-null  int8   
 9   TEMPERATURE_OPTION      60291 non-null  int8   
 10  COST_OF_GOODS_USD       60291 non-null  float64
 11  ITEM_CATEGORY_Beverage  60291 non-null  int32  
 12  ITEM_CATEGORY_Main      60291 non-null  int32  
 13  ITEM_CATEGORY_Dessert   60291 non-null  int32  
 14  ITEM_CATEGORY_Snack     60291 non-null

In [162]:
final_df.corr()

C:\Users\tanko\AppData\Local\Temp\ipykernel_12248\2626907977.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  final_df.corr()


,MENU_ITEM_ID,TRUCK_ID,ORDER_YEAR,ORDER_MONTH,DEMAND,EV_FLAG,DAYS_OPENED,MENU_TYPE_ID,TEMPERATURE_OPTION,COST_OF_GOODS_USD,ITEM_CATEGORY_Beverage,ITEM_CATEGORY_Main,ITEM_CATEGORY_Dessert,ITEM_CATEGORY_Snack
MENU_ITEM_ID,1.000000,0.029032,0.016474,0.003368,0.132419,-0.126577,-0.032044,0.998925,0.035485,0.060110,0.117394,0.070062,-0.346218,-0.059890
TRUCK_ID,0.029032,1.000000,0.025548,-0.003171,0.239371,0.075438,-0.044308,0.029083,-0.006986,0.004133,0.003132,-0.004954,0.008007,-0.004291
ORDER_YEAR,0.016474,0.025548,1.000000,-0.161856,-0.011683,0.308498,-0.540929,0.016692,-0.004068,-0.005345,0.003640,0.000978,-0.000222,-0.010105
ORDER_MONTH,0.003368,-0.003171,-0.161856,1.000000,-0.068573,-0.007347,-0.032374,0.003372,0.000580,0.001230,0.000330,-0.000358,-0.001694,0.001816
DEMAND,0.132419,0.239371,-0.011683,-0.068573,1.000000,0.023086,-0.014647,0.151374,0.574723,0.501136,-0.590446,0.557560,-0.036196,0.118869
EV_FLAG,-0.126577,0.075438,0.308498,-0.007347,0.023086,1.000000,-0.554058,-0.126992,-0.012098,-0.020064,-0.015059,0.008221,0.026077,-0.011620
DAYS_OPENED,-0.032044,-0.044308,-0.540929,-0.032374,-0.014647,-0.554058,1.000000,-0.032443,0.006993,0.008885,-0.006809,-0.001551,0.001480,0.017174
MENU_TYPE_ID,0.998925,0.029083,0.016692,0.003372,0.151374,-0.126992,-0.032443,1.000000,0.061084,0.084060,0.095413,0.095795,-0.349310,-0.064737
TEMPERATURE_OPTION,0.035485,-0.006986,-0.004068,0.000580,0.574723,-0.012098,0.006993,0.061084,1.000000,0.801614,-0.776153,0.847500,-0.205802,0.065245
COST_OF_GOODS_USD,0.060110,0.004133,-0.005345,0.001230,0.501136,-0.020064,0.008885,0.084060,0.801614,1.000000,-0.756692,0.816159,-0.108054,-0.009970


In [163]:
# Scaling

#drop target
target = final_df['DEMAND']
final_df.drop('DEMAND', axis = 1, inplace = True)

scaler = StandardScaler()
scaler.fit(final_df)

final_scaled = scaler.transform(final_df)
final_scaled = pd.DataFrame(final_scaled, columns=final_df.columns)

#assign target variable back to scaled data frame
final_scaled = final_scaled.assign(DEMAND = target)
final_scaled.head()


,MENU_ITEM_ID,TRUCK_ID,ORDER_YEAR,ORDER_MONTH,UNIT_PRICE,EV_FLAG,DAYS_OPENED,MENU_TYPE_ID,TEMPERATURE_OPTION,COST_OF_GOODS_USD,ITEM_CATEGORY_Beverage,ITEM_CATEGORY_Main,ITEM_CATEGORY_Dessert,ITEM_CATEGORY_Snack,DEMAND
0,0.843302,-0.793123,-0.438331,0.402808,-0.810570,-0.549454,0.921407,0.788783,-0.841583,-0.768627,1.084300,-0.871669,-0.244542,-0.234673,7280
1,0.172286,-0.815582,-0.438331,0.402808,-0.810570,-0.549454,2.071937,0.123620,-0.841583,-0.768627,1.084300,-0.871669,-0.244542,-0.234673,7394
2,0.955138,-1.010225,0.833562,-0.807196,2.443052,1.819987,0.921407,1.010504,1.477212,2.760409,-0.922254,1.147224,-0.244542,-0.234673,41661
3,1.178810,-1.002739,0.833562,-0.807196,1.103325,-0.549454,0.057722,1.232226,1.477212,0.878257,-0.922254,1.147224,-0.244542,-0.234673,40743
4,-1.326315,-1.092574,0.833562,-0.807196,-0.810570,-0.549454,-0.515967,-1.428428,-0.841583,-0.847050,-0.922254,-0.871669,4.089281,-0.234673,20285


In [164]:
# Train Test Split
# Define Model Inputs (X) and Output (y)
X = final_scaled.drop('DEMAND',axis=1)
y = final_scaled["DEMAND"]

# Split both Inputs (X) and Ouput (y) into training set (70%) and testing set (30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [165]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

# Create the model "lr"
lr = LinearRegression()

# Fit the model to the training set
lr.fit(X_train, y_train)

# Make predictions on the training and testing sets
train_predictions = lr.predict(X_train)
test_predictions = lr.predict(X_test)

# Calculate RMSE for the training and testing sets
train_rmse = sqrt(mean_squared_error(y_train, train_predictions))
test_rmse = sqrt(mean_squared_error(y_test, test_predictions))

# Calculate R-squared values for the training and testing sets
train_r2 = r2_score(y_train, train_predictions)
test_r2 = r2_score(y_test, test_predictions)

# Print the results
print('Train RMSE:', train_rmse)
print('Test RMSE:', test_rmse)
print('Train R-squared:', train_r2)
print('Test R-squared:', test_r2)


Train RMSE: 11432.27475051986
Test RMSE: 11486.588946075291
Train R-squared: 0.4925646742071531
Test R-squared: 0.48159788159167627


In [167]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 10, random_state = 0)
rf.fit(X_train, y_train)

# Make predictions on the training and testing sets
train_predictions = rf.predict(X_train)
test_predictions = rf.predict(X_test)

# Calculate RMSE for the training and testing sets
train_rmse = sqrt(mean_squared_error(y_train, train_predictions))
test_rmse = sqrt(mean_squared_error(y_test, test_predictions))

# Calculate R-squared values for the training and testing sets
train_r2 = r2_score(y_train, train_predictions)
test_r2 = r2_score(y_test, test_predictions)

# Print the results
print('Train RMSE:', train_rmse)
print('Test RMSE:', test_rmse)
print('Train R-squared:', train_r2)
print('Test R-squared:', test_r2)

Train RMSE: 145.03095223315606
Test RMSE: 292.01970768714716
Train R-squared: 0.9999183348454502
Test R-squared: 0.9996649505328292


In [169]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf, X_train, y_train, cv = 5)
scores.mean()

0.9994346422191718

In [170]:
import xgboost as xg

# create an xgboost regression model
xgb = xg.XGBRegressor()

# Fitting the model
xgb.fit(X_train, y_train)
  
# Make predictions on the training and testing sets
train_predictions = xgb.predict(X_train)
test_predictions = xgb.predict(X_test)

# Calculate RMSE for the training and testing sets
train_rmse = sqrt(mean_squared_error(y_train, train_predictions))
test_rmse = sqrt(mean_squared_error(y_test, test_predictions))

# Calculate R-squared values for the training and testing sets
train_r2 = r2_score(y_train, train_predictions)
test_r2 = r2_score(y_test, test_predictions)

# Print the results
print('Train RMSE:', train_rmse)
print('Test RMSE:', test_rmse)
print('Train R-squared:', train_r2)
print('Test R-squared:', test_r2)

c:\Users\tanko\anaconda3\envs\pysnowpark\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
c:\Users\tanko\anaconda3\envs\pysnowpark\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Train RMSE: 512.1247757588498
Test RMSE: 594.9184496867097
Train R-squared: 0.9989817205835235
Test R-squared: 0.9986094087803329
